# Simple Wave and Wind Hybrid Plant Example
---
In this example, we will walk through the process of simulating a hybrid renewable energy system including both wave and wind energy.

### Import Required Modules
We start by importing the necessary modules and setting up our working environment.

In [1]:
from hopp.simulation import HoppInterface
from hopp.utilities.keys import set_nrel_key_dot_env

/Users/kbrunik/github/forked/HOPP/examples/workshop/log/hybrid_systems_2023-10-25T08.20.44.417405.log


### Set API Key
To access the Wind Toolkit (wind resource) and NSRDB (solar resource) data, we need to set an API key. You can obtain an API key from the [NREL developer website](https://developer.nrel.gov/signup/).

To set up the `NREL_API_KEY` required for resource downloads, you can create an Environment Variable called `NREL_API_KEY`. Otherwise, you can keep the key in a new file called ".env" in the root directory of this project.

Create a file ".env" that contains the single line:
`NREL_API_KEY=key`

In [2]:
# Set API key using the .env
set_nrel_key_dot_env()

#### (Optional)

This step is if you do not set the API key using a ".env" file.

In [ ]:
import os
from hopp.utilities.keys import set_developer_nrel_gov_key

# Set API key manually if not using the .env
global NREL_API_KEY
NREL_API_KEY = os.getenv("NREL_API_KEY") # Set this key manually here if you are not setting it using the .env
set_developer_nrel_gov_key(NREL_API_KEY)  

### Create the HOPP Model
To generate the HOPP Model, instantiate the `HoppInterface` class and supply the required YAML configuration with the technology configuration and site information.

For the site information, the Wind resource data will be downloaded using the API key. The Wave resource data **must be pre-loaded** in the format given in the `Wave_resource_timeseries.csv`.

The Wave technology configuration requires the, device rating (kw), wave power matrix and number of devices.

Note that the wave model doesn't come with a default financial model. To address this, you must establish the `CustomFinancialModel` from HOPP.

The `default_fin_config` contains all of the necessary parameters for the financial calculations.

To maintain consistency across technologies, apply the `default_fin_config` to each technology configuration. This ensures that all technologies use the same financial model throughout the HOPP Model.

In [3]:
hi = HoppInterface("./inputs/06-wave-wind.yaml")

### Add Wave Cost Model Inputs
Add the inputs to run the wave technology cost model and instantiate `MHKCosts`.

In [4]:
cost_model_inputs = {
	'reference_model_num':3,
	'water_depth': 100,
	'distance_to_shore': 80,
	'number_rows': 10,
	'device_spacing':600,
	'row_spacing': 600,
	'cable_system_overbuild': 20
	}
hi.system.wave.create_mhk_cost_calculator(cost_model_inputs)

### Run the Simulation
Simulate the hybrid renewable energy system for a specified number of years (in this case, 25 years).

In [5]:
hi.simulate(25)

### Retrieve and Display Results
Access the simulation results, including annual energies, capacity factors and net present values (NPVs), and print them to the console.

In [6]:
hybrid_plant = hi.system

aeps = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

print("Annual Energy Production")
print(aeps)
print("Net Present Value")
print(npvs)
print("Capacity Factors")
print(cf)

Annual Energy Production
{"wind": 32539658.835298114, "wave": 121325260.0, "hybrid": 106567717.3026002}
Net Present Value
{"wind": -14977200.729403043, "wave": -407735589.40146995, "hybrid": -422077325.4809909}
Capacity Factors
{"wind": 37.14572926403895, "wave": 48.426278059839696, "hybrid": 45.50385609201568}
